### Working with database metadata

- The foundation for these queries are Python objects that represent database concepts like tables and columns
- The most common foundational objects for database metadata in SQLAlchemy are known as MetaData, Table, and Column

### when do Metadata Object has to be made in a program?

```
Having a single MetaData object for an entire application is the most common case, represented as a module-level variable in a single place in an application, often in a “models” or “dbschema” type of package. It is also very common that the MetaData is accessed via an ORM-centric registry or Declarative Base base class, so that this same MetaData is shared among ORM- and Core-declared Table objects.

There can be multiple MetaData collections as well; Table objects can refer to Table objects in other collections without restrictions. However, for groups of Table objects that are related to each other, it is in practice much more straightforward to have them set up within a single MetaData collection, both from the perspective of declaring them, as well as from the perspective of DDL (i.e. CREATE and DROP) statements being emitted in the correct order.
```

### Core Method

In [1]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

Integer, String - these classes represent SQL datatypes and can be passed to a Column with or without necessarily being instantiated. Above, we want to give a length of “30” to the “name” column, so we instantiated String(30). But for “id” and “fullname” we did not specify these, so we can send the class itself.

In [2]:
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)



# to write to dabase, use
`metadata_obj.create_all(engine)`

In [3]:
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"

engine_config = template_engine.format(
        db_username="a",
        db_password="b",
        db_host="-c-db.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="d s")

engine = create_engine(engine_config,
    echo=True
) 

metadata_obj.create_all(engine)

2023-03-06 08:41:57,085 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-06 08:41:57,087 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 08:41:57,102 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-06 08:41:57,103 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 08:41:57,119 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-06 08:41:57,120 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 08:41:57,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 08:41:57,178 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s::VARCHAR(64) COLLATE "C" AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s::VARCHAR, %(param_2)s::VARCHAR, %(param_3)s::VARCHAR, %(param_4)s::VARCHAR, %(param_5)s::VARCHAR]) AND pg_catalog.pg_table_is_visible(pg_

## the ORM way

- When using the ORM, the process by which we declare Table metadata is usually combined with the process of declaring mapped classes
- `DeclarativeBase` comes with an underline `Metadata` object

In [4]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [5]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey
class User(Base):
    
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

### Emitting DDL to the database using an ORM mapping

In [6]:
Base.metadata.create_all(engine)

2023-03-06 08:41:57,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 08:41:57,339 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s::VARCHAR(64) COLLATE "C" AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s::VARCHAR, %(param_2)s::VARCHAR, %(param_3)s::VARCHAR, %(param_4)s::VARCHAR, %(param_5)s::VARCHAR]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s::VARCHAR(64) COLLATE "C"
2023-03-06 08:41:57,339 INFO sqlalchemy.engine.Engine [cached since 0.1615s ago] {'table_name': 'user_account', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-03-06 08:41:57,353 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalo

### Initialitzation __init__() are declared automatively. 

In [10]:

from sqlalchemy.orm import Session

with Session(engine) as session:
    
    sandy = User(name="sandy", fullname="Sandy Cheeks")
    session.add(sandy)
    session.commit()

2023-03-06 08:46:36,265 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 08:46:36,270 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s::VARCHAR(30), %(fullname)s::VARCHAR) RETURNING user_account.id
2023-03-06 08:46:36,271 INFO sqlalchemy.engine.Engine [generated in 0.00162s] {'name': 'sandy', 'fullname': 'Sandy Cheeks'}
2023-03-06 08:46:36,289 INFO sqlalchemy.engine.Engine COMMIT
